<a href="https://colab.research.google.com/github/cZantana/Dataset-random-wikipedia/blob/main/Data_set_500_art%C3%ADculos_random_Wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import re
import pandas as pd
import numpy as np

In [ ]:
titulo = []
articu = []

for i in range(0,500):

  wikirandom='https://es.wikipedia.org/wiki/Special:Random'

  urlrand = requests.get(wikirandom)

  charini=urlrand.text.find('"wgPageName":"')
  charfin=urlrand.text.find('","wgTitle":')

  articulo=urlrand.text[charini+14:charfin]

  if articulo[0]=='"':
    articulo=articulo[1:]
  
  if articulo[-1]==',':
    print("COMAAAA")
    
  linkjson='https://es.wikipedia.org/w/api.php?action=parse&format=json&page='+articulo+'&prop=text&formatversion=2'
  print(i,articulo,"\n")

  try:
    url = requests.get(linkjson)

    newurljson = json.loads(url.text)

    tbody = re.compile('<tbody><tr>(.|\n|\n\n)*?</tr></tbody>')

    encpat = tbody.finditer(str(newurljson['parse']['text']))

    chaon = re.compile('\n')

    tabla = re.compile('<td.*?>')

    tabla2 = re.compile('<th.*?>')

    clean = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

    for noce in encpat:
      #print("noce de cero \n\n",noce[0])
      #print(noce.regs[0][0],noce.regs[0][1])
      #print(type(newurljson['parse']['text'][noce.regs[0][0]:noce.regs[0][1]]))
      tbodyclean = re.sub(chaon,'',noce[0])
      tbodyclean = re.sub(tabla,':\t',tbodyclean)
      tbodyclean = re.sub(tabla2,'\n',tbodyclean)
      tbodyclean = re.sub(clean,'',tbodyclean)
      newurljson['parse']['text']=newurljson['parse']['text'].replace(noce[0],tbodyclean)
      #print("\n\nCambiado\n\n")
      #print(tbodyclean)
      #print(newurljson['parse']['text'])

    clean = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

    htmltext=re.sub(clean, '', newurljson['parse']['text'])

    titulo.append(articulo)
    articu.append(htmltext)
  except:
    print("Error:",Exception)
dataset = pd.DataFrame(list(zip(titulo,articu)),columns=['Titulo','Articulo'])

dataset

In [ ]:
dataset.to_csv('/content/drive/MyDrive/Proyecto Trabajo NLP IA/Articulos.csv', index=True, header=True)

In [ ]:
abrirdataset=pd.read_csv("/content/drive/MyDrive/Proyecto Trabajo NLP IA/Articulos.csv",index_col=0)

datanumpy=abrirdataset.to_numpy()

for i in range(len(datanumpy)):
  print(i)
  numarti=i
  titulo = datanumpy[numarti][0].replace('_',' ')

  #print(datanumpy[numarti][0],"\n")
  #print(titulo)
  #print(datanumpy[numarti][1])
  #print('\n\n\n\n--------------------------------------\n\n\n\n')
  patron = re.compile(fr'.mw-parser-output..*|-->|<!--|↑|\[editar]|\[+[0-9]+\]|\t|Control de autoridades:|Índice(.|\n)*?\n\n\n\n|(.|\n)*?editar datos en Wikidata|(Enlaces externos|Proyectos Wikimedia|Datos: Q2|Referencias|Véase también)(.|\n)*')

  encpat = patron.finditer(str(datanumpy[numarti][1]))

  datanumpy[numarti][1]=re.sub(patron, '', datanumpy[numarti][1])

  '''#print("\n\n\n\nCANTIDAD DE MATCHES:")
  for noce in encpat:
    #print(noce[0])
    #print(noce.regs[0][0],noce.regs[0][1])
    datanumpy[numarti][1]=datanumpy[numarti][1].replace(str(noce[0]),'')'''

  ponertitulo = re.compile(fr'^')
  datanumpy[numarti][1]=re.sub(ponertitulo, titulo+'\n\n', datanumpy[numarti][1])
  #print('------------------------------------------------------------------------------------------------------------------------------------')
  #print(datanumpy[numarti][1])
  #print('------------------------------------------------------------------------------------------------------------------------------------')
  #PARRAFOS:

  parrafos = re.compile(fr'[A-Z](.|\n)*?(\n\n|$)')

  encparr = parrafos.finditer(str(datanumpy[numarti][1]))  

  #print(datanumpy[numarti].shape)
  
  #print(datanumpy[numarti].shape)
  #print(datanumpy[numarti])
  datanumpy=datanumpy.tolist()
  #print('entra for')
  for noce in encparr:
    #print('------------------------------------------------------------------------------------------------------------------------------------')    
    #print(noce[0])
    #print('------------------------------------------------------------------------------------------------------------------------------------')
    #print(noce.regs[0][0],noce.regs[0][1])
    datanumpy[numarti].append(noce[0])
    #print(datanumpy[numarti])
    #print(len(datanumpy[numarti]))
  #print('salefor')
  datanumpy=np.array(datanumpy)
  #print(datanumpy[numarti])

#print(datanumpy[0:1])

#print(datanumpy[numarti][1])

print(len(max(datanumpy, key=len)))
columnas=['Titulo','Articulo']
for i in range(len(max(datanumpy, key=len))-2):
  columnas.append('Parrafo '+str(i+1))

datasetnew=pd.DataFrame(list(datanumpy),columns=columnas)

datasetnew.to_csv('/content/drive/MyDrive/Proyecto Trabajo NLP IA/Parrafos.csv', index=True, header=True)

datasetnew